# Chi-squared test

In [1]:
import numpy as np
import pandas as pd
import scipy as sp
import scipy.stats as stats
import sklearn as sk
import matplotlib.pyplot as plt

In [2]:
import sys
print(sys.version)

3.7.8 (tags/v3.7.8:4b47a5b6ba, Jun 28 2020, 08:53:46) [MSC v.1916 64 bit (AMD64)]


## Chi-squared test
1. 적합도 검정 (Goodness of fit) - 하나의 범주형 변수에 대해 기대하는 어떤 분포를 따르는지 여부를 검정
2. 독립성 검정 (Test of independence) - 한 모집단의 두 개의 범주형 변수가 서로 연관되어 있는지 여부를 검정
3. 동질성 검정 (Test of homogeneity) - 두 모집단의 한 개의 범주형 변수가 서로 분포나 비율이 같은지 여부를 검정

### [예제 1] 어느 공정의 불량율을 유의 수준 5%로 <u>적합도 검정</u> 하시오
출처 : (블로그) 적합도 검정  
https://m.blog.naver.com/PostView.naver?isHttpsRedirect=true&blogId=lchry&logNo=220511965537  
원출처 : 염경철, 염경철 원장의 품질경영기사, 영지문화사

어느 공정의 불량율이 15%이다. 시료를 80개를 추출하여 검사한 결과 불량이 16개이다. 유의 수준 5%로 적합도 검정을 하시오.

가설  
H0 : 불량율 = 15%  
H1 : 시료의 검사 결과가 불량율 15%를 따르지 않는다

In [3]:
result_dict = { 'OK': 64, 'NG': 16 }
crosstab = pd.DataFrame(result_dict, index = ['Measured'])
# 표(교차 집계표)에 합계를 추가한다
crosstab['Sum'] = crosstab.sum(axis=1)
crosstab

,OK,NG,Sum
Measured,64,16,80


In [4]:
defect_rate = 0.15 # 기존 불량율 : 15%
expected_ok = crosstab.iat[0, 2] * (1 - defect_rate)
expected_ng = crosstab.iat[0, 2] * defect_rate
expected_sum = expected_ok + expected_ng
expected_dict = { 'OK': expected_ok, 'NG': expected_ng, 'Sum': expected_sum }
crosstab_temp = pd.DataFrame(expected_dict, index = ['Expected'])
crosstab_temp

,OK,NG,Sum
Expected,68.0,12.0,80.0


In [5]:
crosstab = pd.concat([crosstab, crosstab_temp])
crosstab

,OK,NG,Sum
Measured,64.0,16.0,80.0
Expected,68.0,12.0,80.0


In [6]:
# chi^2를 계산한다
# chi^2 = (관측도수-기대도수)^2/기대도수
chisq_0 = pow(crosstab.iat[0, 0] - crosstab.iat[1, 0], 2) / crosstab.iat[1, 0] + \
            pow(crosstab.iat[0, 1] - crosstab.iat[1, 1], 2) / crosstab.iat[1, 1]

# 유의수준
alpha = 0.05 

# 확률변수(Random Variable)과 임계치를 구한다
rv_chi2 = stats.chi2(1) # 카이제곱분포의 자유도 = 1
chisq_c = rv_chi2.ppf(1-alpha) # chi2.ppf (percent point function) : 분포의 좌측 누적 확률을 인자로 하여 임계치(기각역)를 구한다

# 유의 확률(p-value)도 구해 본다
p_value = 1 - rv_chi2.cdf(chisq_0)

print('Critical Vaule = ', round(chisq_c, 2), ', p-value = ', round(p_value, 2))

Critical Vaule =  3.84 , p-value =  0.21


In [7]:
# chisq_0가 기각역에 포함되면 H0를 기각한다
# 그렇지 않으면 H0를 채택한다 (H0가 틀렸다고는 할 수 없다)
if (chisq_c <= chisq_0):
    print('chisq_c:', round(chisq_c,2), ' <= ', 'chisq_0:', round(chisq_0,2), '[H0 Rejected]')
else:
    print('chisq_c:', round(chisq_c,2), ' > ', 'chisq_0:', round(chisq_0,2), '[H0 Accepted]')

chisq_c: 3.84  >  chisq_0: 1.57 [H0 Accepted]


In [8]:
# 결론 : H0를 채택한다 (H0가 틀렸다고는 할 수 없다)